In [1]:
data = spark.read.parquet("dbfs:///FileStore/tables/stock_file")
data.createOrReplaceTempView("data")
df = spark.sql("select payload.* from data")
df.createOrReplaceTempView("df")

In [2]:
from pyspark.sql.functions import *
df_PushERPStockRequest = spark.sql("""
select PushERPStockRequest.* from df
""")

df_PushERPStockRequest.createOrReplaceTempView("df_PushERPStockRequest")

df_PushERPStockRequest_explode = df_PushERPStockRequest.withColumn('UpdateStockItem', explode('UpdateStockItem'))

df_PushERPStockRequest_explode.createOrReplaceTempView("df_PushERPStockRequest_explode")

In [3]:
df_semi_final = spark.sql("""
select UpdateID,UpdateStockItem.ProductIdentifier.SKU,UpdateStockItem.ProductIdentifier.StoreId,UpdateStockItem.AvailableQuantity,UpdateStockItem.AvailableQuantityActual from df_PushERPStockRequest_explode
""")

df_semi_final.createOrReplaceTempView("df_semi_final")

In [4]:
from datetime import datetime as dt
from datetime import timedelta
from pytz import timezone

offset = dt.now()
offset

Out[196]: datetime.datetime(2020, 10, 3, 8, 57, 33, 897838)

In [5]:
df_final = df_semi_final.withColumn("offSet", lit(offset))
df_final.createOrReplaceTempView("df_final")

In [6]:
df_final = spark.sql("""
select 
first(UpdateID) as UpdateID,
AvailableQuantity,
first(AvailableQuantityActual) as AvailableQuantityActual,
SKU,
StoreId,
max(offset) as offset
from df_final
where offset > (select max(offset) from source_tables_rt.stock)
group by sku, storeid, AvailableQuantity
""")

In [7]:
# df_final.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable("source_tables_rt.table_from_parquet_file")
df_final.write.format("delta").mode("overwrite").saveAsTable("source_tables_rt.table_from_parquet_file")